In [0]:
from IPython.display import clear_output

In [0]:
!pip3 install pycodestyle flake8 pycodestyle_magic
!pip3 install pymorphy2[fast]
clear_output()

In [0]:
%load_ext pycodestyle_magic

## Лекция 2  BM5    

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [0]:
import pickle
import pandas as pd
import numpy as np
from collections import Counter
from collections import defaultdict
from scipy.sparse import csr_matrix
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
### реализуйте эту функцию ранжирования

k = 2.0
b = 0.75


def tf(term, document) -> float:
    return float(document.count(term))


def avgdl(collection) -> float:
    return np.mean(list(map(len, collection)))


def idf(term, collection) -> float:
    n_q = sum([1 for i in range(len(collection)) if term in collection[i]])
    return np.log((len(collection) - n_q + 0.5) / (n_q + 0.5))


def bm25(collection, document, query) -> float:
    return sum([idf(term, collection) * (tf(term, document) * (k + 1)) /
                (tf(term, document) + k * (1 - b + b *
                                           (len(document)) /
                                           (avgdl(collection))))
                for term in query])

### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

In [0]:
!wget https://www.dropbox.com/s/cfjv7galp6ajyr0/quora_question_pairs_rus.csv?dl=0 -O quora_question_pairs_rus.csv
clear_output()

In [6]:
df = pd.read_csv("quora_question_pairs_rus.csv")
df = df.dropna()
df.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [7]:
df.shape

(404267, 4)

In [0]:
queries = list(set(df["question1"]))
query_idx = {queries[i]: i for i in range(len(queries))}
docs = list(set(df["question2"]))
doc_idx = {docs[i]: i for i in range(len(docs))}

In [0]:
df_dup = df[df["is_duplicate"] == 1]
row_ind = df_dup["question1"].apply(lambda x: query_idx[x])
col_ind = df_dup["question2"].apply(lambda x: doc_idx[x])
dup_matrix = csr_matrix((np.ones(df_dup.shape[0]), (row_ind, col_ind)),
                        shape=(len(query_idx), len(doc_idx)))

In [0]:
m = MorphAnalyzer()


def lemmatize(text):
    return [m.parse(word)[0].normal_form 
            for word in simple_word_tokenize(text)]

In [0]:
class SearchEngineBM25():
    def __init__(self, data, b=0.75, k=2):
        self.texts = np.array(data)
        self.b = b
        self.k = k
        self.vectorizer = CountVectorizer(tokenizer=lemmatize)
        tf = self.vectorizer.fit_transform(self.texts)
        N = (tf.getnnz(axis=1)).sum()
        lens = np.array(tf.sum(axis=0))[0]
        self.avgdl = lens.mean()
        lens_rel = np.array([lens[x] / self.avgdl for x in tf.nonzero()[1]])
        idf = np.array([np.log((N - y + 0.5) / (y + 0.5))
                        for y in tf.nonzero()[0]])
        bm25 = idf * tf.data * (k + 1) / (tf.data + k * (1 - b + b * lens_rel))
        self.matrix = csr_matrix((bm25, tf.indices, tf.indptr), shape=tf.shape)
        self.matrix = self.matrix.transpose(copy=True)
        self.tf = []
        df = defaultdict(int)
        for text in self.texts:
            self.tf.append(Counter(lemmatize(text)))
            for word in set(lemmatize(text)):
                df[word] += 1
        self.idf = {key: np.log((N - val + 0.5) / (val + 0.5))
                    for key, val in df.items()}

    def search_naive(self, query, n=5):
        query = lemmatize(query)
        result = []
        for document in self.tf:
            score = 0
            for word in query:
                if word in document:
                    score += self.idf[word] * document[word] * (self.k + 1) / \
                             (document[word] + self.k * (1 - self.b + self.b *
                                                         len(document) /
                                                         self.avgdl))
            result.append(score)
        return sorted(list(zip(self.texts, result)),
                      key=lambda x: x[1], reverse=True)[:n]

    def search_matmul(self, query, n=5):
        query_vec = self.vectorizer.transform([query])
        result = np.array((query_vec * self.matrix).todense())[0]
        indices = np.argsort(result)[::-1].tolist()[:n]
        return list(zip(self.texts[indices], result[indices]))

In [0]:
BMSearchEngine = SearchEngineBM25(docs)

In [13]:
%%time
for query in np.random.choice(np.array(list(df["question1"])), size=100):
    result = BMSearchEngine.search_naive(query)

CPU times: user 4min 6s, sys: 4.38 s, total: 4min 11s
Wall time: 4min 11s


100 запросов обработалось за ≈4 минут, следовательно, 100k запросов обработаются за ≈4000 минут, или ≈67 часов, или ≈3 дня. <br>
Долго.

In [14]:
%%time
for query in np.random.choice(np.array(list(df["question1"])), size=100):
    result = BMSearchEngine.search_matmul(query)

CPU times: user 14.2 s, sys: 44.9 ms, total: 14.2 s
Wall time: 14.2 s


100 запросов обработались за ≈14 секунд, следовательно, 100k запросов обработаются за 14000 секунд, или ≈233 минуты, или ≈4 часа. Уже лучше.

Умножать матрицу на вектор получается в 17 раз быстрее!

### __Задача 2__:    



Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

In [15]:
for result in BMSearchEngine.search_matmul("рождественские каникулы", n=10):
    print(result)

('я в колледже в Индии, могу ли я пойти к филиппинам для своих каникул или после того, как я окончу то, что будет требовать', 8.144191196257989)
('какое лучшее место для посещения во время рождественских каникул из Лондона на 3 - 4 дня для одного путешественника', 6.1509582104618605)
('как насчет летних и зимних каникул в махиндрах эко-хайдарабад', 5.599050265719071)
('каковы некоторые советы по его проведению через процесс собеседования на вакансиях во время каникул marriott по всему миру', 4.752640874140093)
('где лучше всего путешествовать во время каникул', 4.70871951186195)
('как долго проходят рождественские каникулы для университетов в Новой Зеландии', 4.3659420310625)
('какие лучшие вещи делать в летние каникулы', 4.3290825021024375)
('какие рождественские традиции вы и ваша семья имеете на рождественский вечер и рождественский день', 4.314214516048478)
('что самое лучшее, что может сделать студент-механик во время летних каникул, может ли он сделать что-то продуктивное', 4.277

### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1

In [0]:
def accuracy(search_engine, queries, dup_matrix, test_size=10000):
    true_results = 0
    all_results = 0
    test = np.random.choice(queries, size=test_size)
    for query in test:
        if dup_matrix[query_idx[query], ].sum():
            all_results += 1
            results = search_engine.search_matmul(query)
            for result, score in results:
                if dup_matrix[query_idx[query], doc_idx[result]]:
                    true_results += 1
                    break
    return true_results / all_results

b = 0.75

In [17]:
accuracy(BMSearchEngine, queries, dup_matrix)

0.2720812182741117

b = 0

In [0]:
BMSearchEngine15 = SearchEngineBM25(docs, b=0.0)

In [19]:
accuracy(BMSearchEngine15, queries, dup_matrix)

0.058546433378196504

In [0]:
BMSearchEngine11 = SearchEngineBM25(docs, b=1.0)

In [21]:
accuracy(BMSearchEngine11, queries, dup_matrix)

0.24966442953020135

Наилучшее (в 2 раза лучше!) качество поиска достигается при b = 0.

In [0]:
with open("BMSearchEngine.pkl", "wb") as file:
    pickle.dump(BMSearchEngine, file)

In [0]:
from google import colab
colab.files.download("BMSearchEngine.pkl")
clear_output()